## Plate Detection 
Younghoon Cho
### Dataset Used: 
https://www.kaggle.com/datasets/gpiosenka/us-license-plates-image-classification/data

### Plans for Project
1. Normalize from 0 to 1
2. expected size (3, w , h) RGB
3. Model Choice: *Convolutional Neural Network*
4. Train Details
    Data Augmentation : Not acceptable because the states are placed in a certain place. <br>
    Loss Functon : *Cross Entrophy Loss* <br>
    Optimizer : Adam or SGD (haven't chosen yet) <br>
    


In [29]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.nn.functional as F
from torchvision.transforms import v2
from torchvision import datasets
from torch.utils.data import ConcatDataset

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

transforms = v2.Compose([
    v2.RandomResizedCrop(size=(128, 128), antialias=True),
    transforms.ToTensor(),
    v2.Normalize((0.485,), (0.229,)),
])

# download 
train_dir = './archive/plates/train'
train_dir2 = './archive/new plates/train'

valid_dir = './archive/plates/valid'
valid_dir2 = './archive/new plates/valid'

test_dir = './to/archive/plates/test'
test_dir2 = './archive/new plates/test'

# Set random seed for reproducibility
random_state = 1
torch.manual_seed(random_state)

# Create datasets for plates and new plates
train_dataset = datasets.ImageFolder(root=train_dir, transform=transforms)
train_dataset2 = datasets.ImageFolder(root=train_dir2, transform=transforms)

valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transforms)
valid_dataset2 = datasets.ImageFolder(root=valid_dir2, transform=transforms)

test_dataset = datasets.ImageFolder(root=valid_dir, transform=transforms)
test_dataset2 = datasets.ImageFolder(root=valid_dir2, transform=transforms)

full_train_dataset = ConcatDataset([train_dataset, train_dataset2])
full_valid_dataset = ConcatDataset([valid_dataset,valid_dataset2])
full_test_dataset = ConcatDataset([test_dataset,test_dataset2])

# Split train size
original_train_size = len(full_train_dataset)
train_size = int(0.7 * original_train_size)
valid_size = int(0.15 * original_train_size)
test_size = original_train_size - train_size - valid_size

train_dataset, add_valid_dataset, add_test_dataset = random_split(full_train_dataset, [train_size, valid_size, test_size])

full_valid_dataset = ConcatDataset([full_valid_dataset,add_valid_dataset])
full_test_dataset = ConcatDataset([full_test_dataset,add_test_dataset])

# Create data loaders for new plates dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(full_valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(full_test_dataset, batch_size=32, shuffle=False)

# len(train_loader) # 15521 --> 10864
# len(valid_loader) # 530 --> 2858
# len(test_loader)  # 530 --> 2859

10864

### Original Dataset len()
len(train_loader) # 15521
len(valid_loader) # 530
len(test_loader)  # 530

because the dataet is too onesided to train_loader, I have moved 0.15 states each to valid_loader and test_loader

| name of data | previous | after resize data |
|--------------|----------|-------------------|
|train_loader|15521 (93%)| 10864(65.5%)|
|valid_loader|530 (3.5%) | 2858 (17.2%) |
|test_loader|530 (3.5%) | 2859 (17.2%) |

In [37]:
from torchsummary import summary

class plate_recognize_model(nn.Module):
    def __init__(self):
        super().__init__()

        # Model Structure
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3)

        # Pooling layers
        self.pool = nn.MaxPool2d(2, 2)  # MaxPooling2D((2, 2))

        # Fully connected layers
        self.fc1 = nn.Linear(25088 , 256)  # Flatten and Dense(256, activation='relu')
        self.fc2 = nn.Linear(256, 128)  # Dense(128, activation='relu')
        self.fc3 = nn.Linear(128, 56)   # Dense(56, activation='softmax')

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolutional layers with ReLU and pooling
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)

        # Flatten the tensor for fully connected layers
        x = x.view(-1, 25088)  # Flatten from (128 * 16 * 28)

        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = F.relu(self.fc2(x))
        
        # Apply softmax activation in the last layer
        x = F.softmax(self.fc3(x), dim=1)
        
        return x
    
model = plate_recognize_model()

# Print the model summary
print(model)
    
summary(model, input_size=(3,128,128))



plate_recognize_model(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25088, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=56, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 126, 126]           1,792
         MaxPool2d-2           [-1, 64, 63, 63]               0
            Conv2d-3          [-1, 128, 61, 61]          73,856
         MaxPool2d-4          [-1, 128, 30, 30]               0
            Conv2d-5           [-1, 64, 28, 28]          73,792
         MaxPool2d-6       